In [ ]:
# !docker pull qdrant/qdrant
# !docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

In [ ]:
# !pip install qdrant-client

## **Basic Connection and Setup**

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import numpy as np

# Connect to Qdrant
client = QdrantClient(host='localhost', port=6333)
# For cloud Qdrant, use:
# client = QdrantClient(url='https://your-qdrant-cloud-url', api_key='your-api)

# Check if server is running
print(client.get_collections())

collections=[]


## **Create your first collection**

In [3]:
collection_name = "my_collection"

In [ ]:
# Create collection
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=128,  # Dimension of the vectors
        distance=Distance.COSINE  # Distance metric
    ),
)
print(f"Collection '{collection_name}' created.")

Collection 'my_collection' created.


## **Insert vectors**

In [16]:
# Sample vectors (in practice, these would be the embeddings)
vectors = [
    np.random.rand(128).tolist() for _ in range(10)
]

# Sample payloads (optional metadata for each vector)
payloads = [
    {
        "text": f"Document {i}",
        "category": f"{np.random.choice(['science', 'tech', 'art'])}",
        "year": np.random.randint(2000, 2023),
    } for i in range(10)
]

# Insert points into the collection
points = [
    PointStruct(id=i, vector=vectors[i], payload=payloads[i]) for i, (vector, payload) in enumerate(zip(vectors, payloads))
]

client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"Inserted {len(points)} points!")

Inserted 10 points!


## **Search Vectors**

In [56]:
from qdrant_client import models

query_vector = np.random.rand(128).tolist()

search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=5,  # Number of nearest neighbors to return
)

print("Search results\n--------------")
for point in search_results.points:
    print(f"ID: {point.id}, Score: {point.score}, Payload: {point.payload}")

Search results
--------------
ID: 5, Score: 0.7870749, Payload: {'text': 'Document 5', 'category': 'art', 'year': 2008}
ID: 2, Score: 0.7851794, Payload: {'text': 'Document 2', 'category': 'art', 'year': 2005}
ID: 6, Score: 0.7507177, Payload: {'text': 'Document 6', 'category': 'science', 'year': 2014}
ID: 3, Score: 0.75048375, Payload: {'text': 'Document 3', 'category': 'science', 'year': 2005}
ID: 1, Score: 0.74527514, Payload: {'text': 'Document 1', 'category': 'art', 'year': 2014}


## **Filtered Search**

In [73]:
from qdrant_client import models

query_vector = np.random.rand(128).tolist()

filtered_search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="category",
                match=models.MatchAny(any=["science", "tech"])
            ),
            models.FieldCondition(
                key="year",
                range=models.Range(gte=2005, lte=2020)
            )
        ]
    ),
    limit=5,  # Number of nearest neighbors to return
)

print("Filtered search results\n-----------------------")
for point in filtered_search_results.points:
    print(f"ID: {point.id}, Score: {point.score}, Payload: {point.payload}")

Filtered search results
-----------------------
ID: 3, Score: 0.80162513, Payload: {'text': 'Document 3', 'category': 'science', 'year': 2005}
ID: 4, Score: 0.78390145, Payload: {'text': 'Document 4', 'category': 'tech', 'year': 2011}
ID: 0, Score: 0.7705504, Payload: {'text': 'Document 0', 'category': 'tech', 'year': 2014}
ID: 6, Score: 0.7278816, Payload: {'text': 'Document 6', 'category': 'science', 'year': 2014}


## **Batch Operations**

In [82]:
from qdrant_client import models

query_vectors = [
    models.QueryRequest(
        query=np.random.rand(128).tolist()
    ) for _ in range(3)
]

batch_search_results = client.query_batch_points(
    collection_name=collection_name,
    requests=query_vectors,
)

print("Batch search results\n---------------------")
for i, result in enumerate(batch_search_results):
    print(f"Results for query {i+1}:")
    for point in result.points:
        print(f"  ID: {point.id}, Score: {point.score}, Payload: {point.payload}")


Batch search results
---------------------
Results for query 1:
  ID: 1, Score: 0.80270934, Payload: None
  ID: 5, Score: 0.7783747, Payload: None
  ID: 9, Score: 0.7597766, Payload: None
  ID: 6, Score: 0.75165224, Payload: None
  ID: 0, Score: 0.75134635, Payload: None
  ID: 2, Score: 0.74655044, Payload: None
  ID: 7, Score: 0.7423116, Payload: None
  ID: 3, Score: 0.7312421, Payload: None
  ID: 8, Score: 0.7196859, Payload: None
  ID: 4, Score: 0.68680584, Payload: None
Results for query 2:
  ID: 8, Score: 0.8070893, Payload: None
  ID: 7, Score: 0.79090357, Payload: None
  ID: 9, Score: 0.76957846, Payload: None
  ID: 5, Score: 0.7685976, Payload: None
  ID: 1, Score: 0.75433356, Payload: None
  ID: 3, Score: 0.7511483, Payload: None
  ID: 6, Score: 0.7407177, Payload: None
  ID: 4, Score: 0.73720425, Payload: None
  ID: 0, Score: 0.7321327, Payload: None
  ID: 2, Score: 0.731417, Payload: None
Results for query 3:
  ID: 7, Score: 0.78737617, Payload: None
  ID: 4, Score: 0.774495

## **Delete Points**

In [84]:
client.delete(
    collection_name=collection_name,
    points_selector=[1, 2]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

## **Collection Info and Cleanup**

In [87]:
info = client.get_collection(collection_name=collection_name)
print(f"Collection '{collection_name}' info after deletion: {info}")

Collection 'my_collection' info after deletion: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=8 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=128, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, w

In [88]:
# Count points
count = client.count(collection_name=collection_name)
print(f"Number of points in collection '{collection_name}': {count}")

Number of points in collection 'my_collection': count=8


## **Example**

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import numpy as np

# Connect to Qdrant
client = QdrantClient(host='localhost', port=6333)
# For cloud Qdrant, use:
# client = QdrantClient(url='https://your-qdrant-cloud-url', api_key='your-api)

# Check if server is running
print(client.get_collections())

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample documents
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is transforming the world.",
    "Quantum computing is the future of technology.",
    "The Mona Lisa is a masterpiece of art.",
    "Space exploration has advanced significantly in recent years.",
]

# Generate embeddings for the documents
embeddings = model.encode(documents)

# Create collection for text
text_collection = "text_similarity"
client.create_collection(
    collection_name=text_collection,
    vectors_config=VectorParams(
        size=embeddings.shape[1],  # Dimension of the embeddings from the model
        distance=Distance.COSINE  # Distance metric
    ),
)

# Insert documents with embeddings
text_points = [
    PointStruct(id=i, vector=embeddings[i].tolist(), payload={"text": documents[i]}) for i in range(len(documents))
]

client.upsert(
    collection_name=text_collection,
    points=text_points
)

/home/ml3au/anaconda3/envs/vector_db/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [28]:
# Search for similar documents
query_text = "Between AI and quantum computing, which is more transformative?"
query_embedding = model.encode([query_text])[0].tolist()
search_results = client.query_points(
    collection_name=text_collection,
    query=query_embedding,
    limit=2,  # Number of nearest neighbors to return
)

print("Text similarity search results\n-------------------------------")
for point in search_results.points:
    print(f"ID: {point.id}, Score: {point.score}, Text: {point.payload['text']}")

Text similarity search results
-------------------------------
ID: 2, Score: 0.5982033, Text: Quantum computing is the future of technology.
ID: 1, Score: 0.49166825, Text: Artificial intelligence is transforming the world.
